# 利用分类模型预测是否购买新车

<br>

**数据集描述**
背景知识：国外换车比较普遍。每当该款汽车有新车型发布时，人们往往愿意购买。
某汽车公司有400条客户数据，包含客户ID、性别、年龄、工资、新车型发布时是否购买这5个字段。
现在该公司又推出一款新车型，请问销售人员如何从老客户中找到欲购买新车型的客户呢？

**字段解释**
> User ID：字符串类型；客户ID
>
> Gender：字符串类型；性别；有2个值，分别是Male（男性）和Female（女性）
>
> Age：数值型；年龄
>
> EstimatedSalary：数值型；预估的客户工资
> 
> Purchased：字符串类型；新车型发布时是否购买；有2个值，分别是0（每购买）和1（购买了）


## 导入库

In [22]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

## 导入数据集

In [8]:
data=pd.read_csv("./data/Social_Network_Ads.csv")
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB


,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


## 数据预处理

In [9]:
### 删除无关列 NO.
data.drop(['User ID'],axis=1,inplace=True)
data.head()

,Gender,Age,EstimatedSalary,Purchased
0,Male,19,19000,0
1,Male,35,20000,0
2,Female,26,43000,0
3,Female,27,57000,0
4,Male,19,76000,0


In [10]:
### 填充空缺值 - 无空缺值
def calcNull(data):
    nullSum=data.isnull().sum()
    nullSum=nullSum.drop(nullSum[nullSum.values==0].index)
    return nullSum
calcNull(data)

Series([], dtype: int64)

In [13]:
### 将Purchased字段改为类别类型
data['Purchased']=data['Purchased'].apply(str)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Gender           400 non-null    object
 1   Age              400 non-null    int64 
 2   EstimatedSalary  400 non-null    int64 
 3   Purchased        400 non-null    object
dtypes: int64(2), object(2)
memory usage: 12.6+ KB


In [26]:
## 分离自变量因变量
X=data.iloc[:,:-1]
# X.head()
Y=data.iloc[:,-1]
Y.head()

0    0
1    0
2    0
3    0
4    0
Name: Purchased, dtype: object

In [27]:
## 特征缩放
LE=LabelEncoder()
X["Gender"]=LE.fit_transform(X["Gender"])
SC= StandardScaler()
X=SC.fit_transform(X)

array([[ 1.02020406, -1.78179743, -1.49004624],
       [ 1.02020406, -0.25358736, -1.46068138],
       [-0.98019606, -1.11320552, -0.78528968],
       ...,
       [-0.98019606,  1.17910958, -1.46068138],
       [ 1.02020406, -0.15807423, -1.07893824],
       [-0.98019606,  1.08359645, -0.99084367]])

In [28]:
## 拆分训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)


(280, 3) (120, 3) (280,) (120,)


## 使用KNN进行训练

### sklearn.neighbors.KNeighborsClassifier 简单解释

``` python
class sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, weights=’uniform’, 
											algorithm=’auto’, leaf_size=30, 
											p=2, metric=’minkowski’, 
											metric_params=None, 
											n_jobs=None, **kwargs)
```
[api详解](https://www.cnblogs.com/pinard/p/6065607.html)
> - **n_neighbors** ： int，optional(default = 5)
> 默认情况下kneighbors查询使用的邻居数。就是k-NN的k的值，选取最近的k个点。
>
> - **weights** ： str或callable，可选(默认=‘uniform’)
> 默认是uniform，参数可以是uniform、distance，也可以是用户自己定义的函数。uniform是均等的权重，就说所有的邻近点的权重都是相等的。distance是不均等的权重，距离近的点比距离远的点的影响大。用户自定义的函数，接收距离的数组，返回一组维数相同的权重。
> 
> - **algorithm** ： {‘auto’，‘ball_tree’，‘kd_tree’，‘brute’}，可选
> 快速k近邻搜索算法，默认参数为auto，可以理解为算法自己决定合适的搜索算法。除此之外，用户也可以自己指定搜索算法ball_tree、kd_tree、brute方法进行搜索，brute是蛮力搜索，也就是线性扫描，当训练集很大时，计算非常耗时。kd_tree，构造kd树存储数据以便对其进行快速检索的树形数据结构，kd树也就是数据结构中的二叉树。以中值切分构造的树，每个结点是一个超矩形，在维数小于20时效率高。ball tree是为了克服kd树高纬失效而发明的，其构造过程是以质心C和半径r分割样本空间，每个节点是一个超球体。
> 
> - **leaf_size** ： int，optional(默认值= 30)
> 默认是30，这个是构造的kd树和ball树的大小。这个值的设置会影响树构建的速度和搜索速度，同样也影响着存储树所需的内存大小。需要根据问题的性质选择最优的大小。
> 
> - **p** ： 整数，可选(默认= 2) 
> p是使用距离度量参数 metric 附属参数，只用于闵可夫斯基距离和带权重闵可夫斯基距离中p值的选择，p=1为曼哈顿距离， p=2为欧式距离。默认为2
> 
> - **metric** ： 字符串或可调用，默认为’minkowski’
> 用于距离度量，默认度量是minkowski，也就是p=2的欧氏距离(欧几里德度量)。
> 
> - **metric_params** ： dict，optional(默认=None)
> 距离公式的其他关键参数，这个可以不管，使用默认的None即可。
> 
> - **n_jobs** ： int或None，可选(默认=None)
> 并行处理设置。默认为1，临近点搜索并行工作数。如果为-1，那么CPU的所有cores都用于并行工作。

In [58]:
### 构建并训练模型
from sklearn.neighbors import KNeighborsClassifier
cf = KNeighborsClassifier(n_neighbors = 5)
cf.fit(X_train, Y_train)

KNeighborsClassifier()

In [59]:
### 预测模型
Y_pre=cf.predict(X_test)
Y_pre

array(['0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '1',
       '1', '1', '0', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0',
       '1', '1', '0', '0', '0', '1', '1', '0', '1', '0', '0', '1', '1',
       '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0',
       '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '1', '1', '1',
       '1', '1', '0', '1', '0', '0', '1', '0', '0', '1', '1', '0', '1',
       '0', '0', '1', '1', '1', '0', '1', '0', '0', '0', '1', '0', '0',
       '1', '0', '1', '0', '1', '0', '0', '1', '1', '0', '1', '0', '0',
       '0', '1', '1', '0', '0', '0', '1', '1', '0', '0', '0', '1', '0',
       '0', '0', '0'], dtype=object)

In [60]:
### 生成混淆矩阵
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pre)
print("混淆矩阵:",cm)
print("准确度:",(cm[0][0]+cm[1][1])/cm.sum())

混淆矩阵: [[70  7]
 [ 3 40]]
准确度: 0.9166666666666666


In [67]:
### 挑选最优参
def search_best_neighbors(st,ed):
    best_score=0
    best_i=0;
    for i in range(st,ed+1):
        cf = KNeighborsClassifier(n_neighbors = i)
        cf.fit(X_train, Y_train)
        Y_pre=cf.predict(X_test)
        Y_pre
        cm = confusion_matrix(Y_test, Y_pre)
        print("--------n_neighbors="+str(i)+"--------")
        print("准确度:",(cm[0][0]+cm[1][1])/cm.sum())
        print("混淆矩阵:",cm)
        if((cm[0][0]+cm[1][1])/cm.sum()>best_score):
            best_i=i
            best_score=(cm[0][0]+cm[1][1])/cm.sum()
    return best_i,best_score
        
best_i,best_score=search_best_neighbors(1,30)
print("最优的neighbors为%d,准确度为%.3f"%(best_i,best_score))

--------n_neighbors=1--------
准确度: 0.8583333333333333
混淆矩阵: [[66 11]
 [ 6 37]]
--------n_neighbors=2--------
准确度: 0.8666666666666667
混淆矩阵: [[72  5]
 [11 32]]
--------n_neighbors=3--------
准确度: 0.9083333333333333
混淆矩阵: [[70  7]
 [ 4 39]]
--------n_neighbors=4--------
准确度: 0.9166666666666666
混淆矩阵: [[71  6]
 [ 4 39]]
--------n_neighbors=5--------
准确度: 0.9166666666666666
混淆矩阵: [[70  7]
 [ 3 40]]
--------n_neighbors=6--------
准确度: 0.9166666666666666
混淆矩阵: [[71  6]
 [ 4 39]]
--------n_neighbors=7--------
准确度: 0.9
混淆矩阵: [[68  9]
 [ 3 40]]
--------n_neighbors=8--------
准确度: 0.9083333333333333
混淆矩阵: [[70  7]
 [ 4 39]]
--------n_neighbors=9--------
准确度: 0.9083333333333333
混淆矩阵: [[69  8]
 [ 3 40]]
--------n_neighbors=10--------
准确度: 0.9083333333333333
混淆矩阵: [[69  8]
 [ 3 40]]
--------n_neighbors=11--------
准确度: 0.9083333333333333
混淆矩阵: [[69  8]
 [ 3 40]]
--------n_neighbors=12--------
准确度: 0.8916666666666667
混淆矩阵: [[70  7]
 [ 6 37]]
--------n_neighbors=13--------
准确度: 0.8916666666666667
混淆矩阵: [[7

In [69]:
# 使用朴素贝叶斯模型进行训练
from sklearn.naive_bayes import GaussianNB
cf = GaussianNB()
cf.fit(X_train, Y_train)
### 预测模型
Y_pre=cf.predict(X_test)
Y_pre
### 生成混淆矩阵
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pre)
print("混淆矩阵:",cm)
print("准确度:",(cm[0][0]+cm[1][1])/cm.sum())

混淆矩阵: [[69  8]
 [ 7 36]]
准确度: 0.875


## 使用 LogisticRegression逻辑回归模型

```python
class sklearn.linear_model.LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
```
[api参数详解](https://blog.csdn.net/jark_/article/details/78342644)

> penalty：惩罚项，str类型，可选参数为l1和l2，默认为l2。用于指定惩罚项中使用的规范。newton-cg、sag和lbfgs求解算法只支持L2规范。L1G规范假设的是模型的参数满足拉普拉斯分布，L2假设的模型参数满足高斯分布，所谓的范式就是加上对参数的约束，使得模型更不会过拟合(overfit)，但是如果要说是不是加了约束就会好，这个没有人能回答，只能说，加约束的情况下，理论上应该可以获得泛化能力更强的结果。
>
> dual：对偶或原始方法，bool类型，默认为False。对偶方法只用在求解线性多核(liblinear)的L2惩罚项上。当样本数量>样本特征的时候，dual通常设置为False。
>
> tol：停止求解的标准，float类型，默认为1e-4。就是求解到多少的时候，停止，认为已经求出最优解。
>
> c：正则化系数λ的倒数，float类型，默认为1.0。必须是正浮点型数。像SVM一样，越小的数值表示越强的正则化。
>
> fit_intercept：是否存在截距或偏差，bool类型，默认为True。
>
> intercept_scaling：仅在正则化项为”liblinear”，且fit_intercept设置为True时有用。float类型，默认为1。
>
> class_weight：用于标示分类模型中各种类型的权重，可以是一个字典或者’balanced’字符串，默认为不输入，也就是不考虑权重，即为None。如果选择输入的话，可以选择balanced让类库自己计算类型权重，或者自己输入各个类型的权重。举个例子，比如对于0,1的二元模型，我们可以定义class_weight={0:0.9,1:0.1}，这样类型0的权重为90%，而类型1的权重为10%。如果class_weight选择balanced，那么类库会根据训练样本量来计算权重。某种类型样本量越多，则权重越低，样本量越少，则权重越高。当class_weight为balanced时，类权重计算方法如下：n_samples / (n_classes * np.bincount(y))。n_samples为样本数，n_classes为类别数量，np.bincount(y)会输出每个类的样本数，例如y=[1,0,0,1,1],则np.bincount(y)=[2,3]。
>
> random_state：随机数种子，int类型，可选参数，默认为无，仅在正则化优化算法为sag,liblinear时有用。
>
> solver：优化算法选择参数，只有五个可选参数，即newton-cg,lbfgs,liblinear,sag,saga。默认为liblinear。solver参数决定了我们对逻辑回归损失函数的优化方法，有四种算法可以选择，分别是：
> - liblinear：使用了开源的liblinear库实现，内部使用了坐标轴下降法来迭代优化损失函数。
> - lbfgs：拟牛顿法的一种，利用损失函数二阶导数矩阵即海森矩阵来迭代优化损失函数。
> - newton-cg：也是牛顿法家族的一种，利用损失函数二阶导数矩阵即海森矩阵来迭代优化损失函数。
> - sag：即随机平均梯度下降，是梯度下降法的变种，和普通梯度下降法的区别是每次迭代仅仅用一部分的样本来计算梯度，适合于样本数据多的时候。
> - saga：线性收敛的随机优化算法的的变重。
>
> max_iter：算法收敛最大迭代次数，int类型，默认为10。仅在正则化优化算法为newton-cg, sag和lbfgs才有用，算法收敛的最大迭代次数。
>
> multi_class：分类方式选择参数，str类型，可选参数为ovr和multinomial，默认为ovr。ovr即前面提到的one-vs-rest(OvR)，而multinomial即前面提到的many-vs-many(MvM)。如果是二元逻辑回归，ovr和multinomial并没有任何区别，区别主要在多元逻辑回归上。
>
> verbose：日志冗长度，int类型。默认为0。就是不输出训练过程，1的时候偶尔输出结果，大于1，对于每个子模型都输出。
>
> warm_start：热启动参数，bool类型。默认为False。如果为True，则下一次训练是以追加树的形式进行（重新使用上一次的调用作为初始化）。
>
> n_jobs：并行数。int类型，默认为1。1的时候，用CPU的一个内核运行程序，2的时候，用CPU的2个内核运行程序。为-1的时候，用所有CPU的内核运行程序。


In [71]:
# 构建逻辑回归模型并训练模型
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression( class_weight='balanced')
classifier.fit(X_train, Y_train)
### 预测模型
Y_pre=cf.predict(X_test)
Y_pre
### 生成混淆矩阵
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pre)
print("混淆矩阵:",cm)
print("准确度:",(cm[0][0]+cm[1][1])/cm.sum())

混淆矩阵: [[69  8]
 [ 7 36]]
准确度: 0.875
